In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Step 1: Generate synthetic spatial features and initial urban area
def generate_synthetic_data(size=100):
    np.random.seed(0)
    slope = np.random.rand(size, size)
    distance_to_road = np.random.rand(size, size)
    population_density = np.random.rand(size, size)

    # Initial urban region in the center
    urban = np.zeros((size, size))
    urban[size//2 - 5:size//2 + 5, size//2 - 5:size//2 + 5] = 1

    return slope, distance_to_road, population_density, urban

# Step 2: Prepare training data
def prepare_training_data(slope, road, pop, urban):
    X = np.stack([slope, road, pop], axis=-1).reshape(-1, 3)
    y = urban.flatten()
    return X, y

# Step 3: Train Random Forest model
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    model = RandomForestClassifier(n_estimators=50, random_state=0)
    model.fit(X_train, y_train)
    return model

# Step 4: Cellular Automata Simulation
def get_neighboring_urban_cells(urban_map, x, y):
    count = 0
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx == 0 and dy == 0:
                continue
            nx, ny = x + dx, y + dy
            if 0 <= nx < urban_map.shape[0] and 0 <= ny < urban_map.shape[1]:
                count += urban_map[nx, ny]
    return count

def ca_simulation(urban, features, model, iterations=5):
    size = urban.shape[0]
    for _ in range(iterations):
        updated_urban = urban.copy()
        for x in range(size):
            for y in range(size):
                if urban[x, y] == 0:
                    neighbors = get_neighboring_urban_cells(urban, x, y)
                    if neighbors >= 2:  # Threshold condition
                        feat = features[x, y].reshape(1, -1)
                        if model.predict(feat) == 1:
                            updated_urban[x, y] = 1
        urban = updated_urban
    return urban

# Step 5: Visualization
def visualize(initial, future):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.title("Initial Urban Area")
    plt.imshow(initial, cmap="gray")
    plt.subplot(1, 2, 2)
    plt.title("Simulated Urban Expansion")
    plt.imshow(future, cmap="gray")
    plt.show()

# Run the full pipeline
slope, road, pop, urban = generate_synthetic_data()
X, y = prepare_training_data(slope, road, pop, urban)
model = train_model(X, y)

features = np.stack([slope, road, pop], axis=-1)
urban_future = ca_simulation(urban, features, model, iterations=5)
visualize(urban, urban_future)
